# TD 4: Mitigation des biais avec des méthodes de pré-processing et de post-processing



## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings
  The next cell of code are to execute only once per colab environment


#### Python env creation

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```
### Local Settings

#### 1. Uv installation


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`


#### 3. Python env creation

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```


In [14]:
# To execute only in Colab
# ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

In [ ]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

## 1.Manipulate the dataset

In [15]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score


MEPSDataset19_data = MEPSDataset19()
(dataset_orig_panel19_train, dataset_orig_panel19_val, dataset_orig_panel19_test) = (
    MEPSDataset19().split([0.5, 0.8], shuffle=True)
)

In [16]:
len(dataset_orig_panel19_train.instance_weights), len(
    dataset_orig_panel19_val.instance_weights
), len(dataset_orig_panel19_test.instance_weights)

(7915, 4749, 3166)

In [17]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights

array([21854.981705, 18169.604822, 17191.832515, ...,  3896.116219,
        4883.851005,  6630.588948], shape=(15830,))

In [18]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

'Taille du dataset 15830, poids total du dataset 141367240.546316.'

Conversion en dataframe

In [19]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df["WEIGHT"] = data[1]["instance_weights"]
    return df


df = get_df(MEPSDataset19_data)

Nous réalisons maintenant l'opération inverse (qui sera indispensable pour le projet). Créer un objet de la classe StandardDataset de AIF360 à partir du dataframe. 

Pour le projet cela vous permettre d'utiliser les méthode déjà implémentées dans AIF360 sur votre jeu de données.

Ici cela n'a aucun intéret car le dataframe vien d'un StandardDataset, nous vous fournissons le code. Mais cela vaut le coup de le lire attentivement et de poser des questions si besoin.




In [20]:
import os
from aif360.datasets import StandardDataset
import pandas as pd

# Get categorical column from one hot encoding (specitic to MEPSdataset)
# Here we create a dictionnary that links each categorical column name
# to the list of corresponding one hot encoded columns
categorical_columns_dic = {}
for col in df.columns:
    col_split = col.split("=")
    if len(col_split) > 1:
        cat_col = col_split[0]
        if not (cat_col in categorical_columns_dic.keys()):
            categorical_columns_dic[cat_col] = []
        categorical_columns_dic[cat_col].append(col)
categorical_features = categorical_columns_dic.keys()

In [21]:
# Now we recreate the categorical column value from the one hot encoded
print(df.shape)


def categorical_transform(df, onehotencoded, cat_col):
    if len(onehotencoded) > 1:
        return df[onehotencoded].apply(
            lambda x: onehotencoded[np.argmax(x)][len(cat_col) + 1 :], axis=1
        )
    else:
        return df[onehotencoded]


# Reverse the categorical one hot encoded
for cat_col, onehotencoded in categorical_columns_dic.items():
    df[cat_col] = categorical_transform(df, onehotencoded, cat_col)
    df.drop(columns=onehotencoded, inplace=True)

df.shape

(15830, 140)


(15830, 43)

In [22]:
MyDataset = StandardDataset(
    df=df,
    label_name="UTILIZATION",
    favorable_classes=[1],
    protected_attribute_names=["RACE"],
    privileged_classes=[[1]],
    instance_weights_name="WEIGHT",
    categorical_features=categorical_features,
    features_to_keep=[],
    features_to_drop=[],
    na_values=["?", "Unknown/Invalid"],
    custom_preprocessing=None,
    metadata=None,
)

In [23]:
# We check the dataset has the same metrics :D
# Attention étonnanement le positive label 'favorable_classes' est par défaut 1 (cela est un peu bizarre pour ce dataset)
print(
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{"RACE": 0}],
        privileged_groups=[{"RACE": 1}],
    ).base_rate(),
)
print(
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).disparate_impact(),
    BinaryLabelDatasetMetric(
        MyDataset, unprivileged_groups=[{"RACE": 0}], privileged_groups=[{"RACE": 1}]
    ).base_rate(),
)

0.49826823461176517 0.21507139363038463
0.49826823461176517 0.21507139363038463


In [24]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate

dir = disparate_impact_ratio(
    y_true=df.UTILIZATION, prot_attr=df.RACE, pos_label=1, sample_weight=df.WEIGHT
)
br = base_rate(y_true=df.UTILIZATION, pos_label=1, sample_weight=df.WEIGHT)
dir, br

(0.4982682346117653, np.float64(0.21507139363038463))

## 2. Appliquer les méthodes de pré-processing disponibles dans AIF360

In [27]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]
]
privileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]
]
sens_attr, unprivileged_groups, privileged_groups

('RACE', [{'RACE': np.float64(0.0)}], [{'RACE': np.float64(1.0)}])

### 2.1 Quesiton: Apprendre une regression logistique qui prédit l'UTILIZATION

Attention nous avons enlever le preprocessing sur le dataframe, il faut cette fois utiliser l'API d'AIF360
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StructuredDataset.html

pour retrouver les features (X), les labels (y) et les poids de chaque instance du dataset

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

X_train = dataset_orig_panel19_train.features
y_train = dataset_orig_panel19_train.labels[:,0]
X_val = dataset_orig_panel19_val.features
y_val = dataset_orig_panel19_val.labels[:,0]


model = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model = model.fit(
    X_train,
    y_train,
    **{"logisticregression__sample_weight": dataset_orig_panel19_train.instance_weights}
)

preds = model.predict(X_val)

model.score(X_val, y_val, sample_weight=dataset_orig_panel19_val.instance_weights)

0.8395130386823652

In [29]:
accuracy_score(y_val, preds, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8395130386823652, 0.708840349124986)

### 2.2 Question: Calcul des métriques de fairness

Calculer les métriques du dataset de validation seul.

Calculer les métriques basées sur les prédictions et la vérité du dataset de validation.

En comparaison calculer les métriques basées sur des prédictions aléatoires et la vérité du dataset de validation.

In [30]:
# Metrics on validation dataset
get_metrics(
        y_true=y_val,
        y_pred=None,
        prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
        pos_label=1,
        sample_weight=dataset_orig_panel19_val.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.13584669171021266),
 'disparate_impact_ratio': 0.4910855710688648}

In [31]:
# Metrics based on predictions and truth on validation dataset
get_metrics(
        y_true=y_val,
        y_pred=preds,
        prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
        pos_label=1,
        sample_weight=dataset_orig_panel19_val.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.14093443635350933),
 'disparate_impact_ratio': 0.3198008242753057,
 'base_rate_preds': np.float64(0.15214409985729335),
 'equal_opportunity_difference': -0.20724951359870203,
 'average_odds_difference': -0.13465510387778157,
 'conditional_demographic_disparity': np.float64(-0.056892160491250884),
 'smoothed_edf': np.float64(1.140056560441825),
 'df_bias_amplification': np.float64(0.42891980941410734),
 'balanced_accuracy_score': 0.708840349124986}

In [32]:
# Metrics based on random predictions and truth on validation dataset

pred_random = np.array([int(r*2) for r in np.random.random(size=len(y_val)).tolist()])
pred_random.max(), pred_random.min()
get_metrics(
        y_true=y_val,
        y_pred=pred_random,
        prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
        pos_label=1,
        sample_weight=dataset_orig_panel19_val.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(0.045059590006641004),
 'disparate_impact_ratio': 1.095834904133213,
 'base_rate_preds': np.float64(0.48778047281512177),
 'equal_opportunity_difference': 0.06340701258640247,
 'average_odds_difference': 0.05064364360085588,
 'conditional_demographic_disparity': np.float64(0.00939115827286097),
 'smoothed_edf': np.float64(0.09151653802208826),
 'df_bias_amplification': np.float64(-0.6196202130056293),
 'balanced_accuracy_score': 0.48530498218618456}

### 2.2 Repondération
#### 2.2.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire de repondération et les appliquer sur le dataset d'apprentissage

In [33]:
from aif360.algorithms.preprocessing import *

RW = Reweighing(
    unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups
)

In [34]:
RW.fit(dataset_orig_panel19_train)
dataset_transf_train = RW.transform(dataset_orig_panel19_train)
dataset_transf_val = RW.transform(dataset_orig_panel19_val)

#### 2.2.2. Question: Apprendre une regression logistique sur les données pondérées et calculer les métriques de fairness sur l'échantillon de validation

Comme vu en cours le Reweighting ne modifie que la pondération du dataset, les features et label restent inchangés.

In [35]:
model_rw = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model_rw = model_rw.fit(
    X_train,
    y_train,
    **{"logisticregression__sample_weight": dataset_transf_train.instance_weights}
)

preds_rw = model_rw.predict(X_val)

model_rw.score(X_val, y_val), model_rw.score(X_val, y_val, sample_weight=dataset_transf_val.instance_weights)

(0.8612339439882081, 0.8330705544304328)

In [36]:
accuracy_score(y_val, preds_rw, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds_rw, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8326581068729562, 0.6886110023890155)

In [37]:
# Metrics on prediction with RW on validation dataset
get_metrics(
    y_true=y_val,
    y_pred=preds_rw,
    prot_attr=dataset_orig_panel19_val.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=dataset_transf_val.instance_weights,
)

{'base_rate_truth': np.float64(0.21226910887324338),
 'statistical_parity_difference': np.float64(-0.014255553664911602),
 'disparate_impact_ratio': 0.9024495192759333,
 'base_rate_preds': np.float64(0.14057984998837425),
 'equal_opportunity_difference': 0.007035005711166775,
 'average_odds_difference': -0.00590625240601339,
 'conditional_demographic_disparity': np.float64(-0.006190961230538837),
 'smoothed_edf': np.float64(0.10264244923839327),
 'df_bias_amplification': np.float64(0.09153838089694943),
 'balanced_accuracy_score': 0.6887406389747808}

### 2.3. Disparate Impact Remover
#### 2.3.1. Question : Trouver dans l'API quels objets/fonctions sont à utiliser pour faire une approache de disparate impact remover et les appliquer. 


In [38]:
DIR = DisparateImpactRemover(repair_level=1., sensitive_attribute='RACE')
train_dir = DIR.fit_transform(dataset_orig_panel19_train)
val_dir = DIR.fit_transform(dataset_orig_panel19_val)

In [39]:
dataset_orig_panel19_train

               instance weights features                                    \
                                         protected attribute                 
                                     AGE                RACE  PCS42  MCS42   
instance names                                                               
2301                7146.545007     55.0                 0.0  57.49  54.20   
9495                4107.337349     38.0                 0.0  54.22  55.39   
9027               15116.392383     63.0                 1.0  51.26  35.33   
9168                3341.885401     18.0                 0.0  42.40  63.27   
8753                3556.883879     18.0                 0.0  56.71  62.39   
...                         ...      ...                 ...    ...    ...   
41                 19250.951023     54.0                 1.0  56.15  57.16   
13297               2626.412576     39.0                 0.0  50.30  55.61   
4006                5081.599952     85.0                 0.0  20

In [40]:
train_dir

               instance weights features                                    \
                                         protected attribute                 
                                     AGE                RACE  PCS42  MCS42   
instance names                                                               
2301                7146.545007     55.0                 0.0  57.49  54.19   
9495                4107.337349     38.0                 0.0  54.22  55.39   
9027               15116.392383     63.0                 1.0  50.73  34.87   
9168                3341.885401     18.0                 0.0  41.42  63.27   
8753                3556.883879     18.0                 0.0  56.71  62.36   
...                         ...      ...                 ...    ...    ...   
41                 19250.951023     54.0                 1.0  55.35  56.70   
13297               2626.412576     39.0                 0.0  50.28  55.61   
4006                5081.599952     85.0                 0.0  19

#### 2.3.2. Question: Apprendre une regression logistique sur les données transformées en retirant l'attribut sensible et calculer les métriques de fairness sur l'échantillon de validation

In [41]:
protected="RACE"
index = dataset_orig_panel19_train.feature_names.index(protected)
model_dir = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model_dir = model_dir.fit(
    np.delete(train_dir.features, index, axis=1),
    train_dir.labels[:,0],
    **{"logisticregression__sample_weight": train_dir.instance_weights}
)

preds_dir = model_dir.predict(np.delete(val_dir.features, index, axis=1))

model_dir.score(np.delete(val_dir.features, index, axis=1), val_dir.labels[:,0], sample_weight=val_dir.instance_weights)

0.8400100853792806

In [42]:
accuracy_score(y_val, preds_dir, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds_dir, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8400100853792806, 0.7053203897748427)

In [43]:
get_metrics(
    y_true=val_dir.labels[:,0],
    y_pred=preds_dir,
    prot_attr=val_dir.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=val_dir.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.11329316954757428),
 'disparate_impact_ratio': 0.4080612722495058,
 'base_rate_preds': np.float64(0.14713888821048746),
 'equal_opportunity_difference': -0.15186627250442014,
 'average_odds_difference': -0.09608825241172816,
 'conditional_demographic_disparity': np.float64(-0.04701218213820861),
 'smoothed_edf': np.float64(0.8963376749045708),
 'df_bias_amplification': np.float64(0.18520092387685327),
 'balanced_accuracy_score': 0.7053203897748427}

### 2.4. Question: Apprentissage de représentation latente fair

Apprendre le pre-processing et evaluer son impact avec les métriques

In [44]:
from aif360.algorithms.preprocessing.lfr import LFR
TR = LFR(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    k=5,
    Ax=0.01,
    Ay=1.0,
    Az=50.0,
    print_interval=250,
    verbose=1,
    seed=None,
)
TR = TR.fit(dataset_orig_panel19_train, maxiter=5000, maxfun=5000)

step: 0, loss: 1.69019295342257, L_x: 73.81000902947056,  L_y: 0.7422937630397612,  L_z: 0.004195982001762061
step: 250, loss: 1.6901930202302262, L_x: 73.810009051626,  L_y: 0.7422937683058747,  L_z: 0.004195983228161832
step: 500, loss: 1.6901928993149677, L_x: 73.81000903010553,  L_y: 0.7422937709301678,  L_z: 0.004195980761674894
step: 750, loss: 1.4936821803790545, L_x: 73.7777970675713,  L_y: 0.4735322920457071,  L_z: 0.005647438353152684
step: 1000, loss: 1.4936821601096708, L_x: 73.77779711332491,  L_y: 0.47353229387508383,  L_z: 0.005647437902026759
step: 1250, loss: 1.493682187894152, L_x: 73.77779708111517,  L_y: 0.47353229002863934,  L_z: 0.005647438541087219
step: 1500, loss: 1.3349354346450175, L_x: 73.80464716315878,  L_y: 0.46430653993217974,  L_z: 0.0026516484616250003
step: 1750, loss: 1.3349352071609666, L_x: 73.8046471783519,  L_y: 0.464306548171032,  L_z: 0.0026516437441283123
step: 2000, loss: 1.3349355032783756, L_x: 73.80464714460902,  L_y: 0.4643065428863328,  

In [45]:
# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_panel19_train)
dataset_transf_val = TR.transform(dataset_orig_panel19_val)

In [46]:
model_lfr = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))

model_lfr = model_lfr.fit(
    dataset_transf_train.features,
    y_train,
    **{"logisticregression__sample_weight": dataset_transf_train.instance_weights}
)

preds_lfr = model_lfr.predict(dataset_transf_val.features)

model_lfr.score(dataset_transf_val.features, y_val, sample_weight=dataset_transf_val.instance_weights)

0.8078534797262537

In [47]:
accuracy_score(y_val, preds_lfr, sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, preds_lfr, sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8078534797262537, 0.5726977338647012)

In [48]:
get_metrics(
            y_true=y_val,
            y_pred=preds_lfr,
            prot_attr=dataset_transf_val.protected_attributes[:, sens_ind],
            pos_label=1,
            sample_weight=dataset_transf_val.instance_weights,
        )

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.018138050641821286),
 'disparate_impact_ratio': 0.667548925004786,
 'base_rate_preds': np.float64(0.04747348034760007),
 'equal_opportunity_difference': -0.00408179662840899,
 'average_odds_difference': -0.0006331560634439265,
 'conditional_demographic_disparity': np.float64(-0.020886945234518745),
 'smoothed_edf': np.float64(0.40414214278625726),
 'df_bias_amplification': np.float64(-0.3069946082414603),
 'balanced_accuracy_score': 0.5726977338647012}

## 3 Post processing

### 3.1 Question: Use the post-processing Reject Option Classification

In [49]:
from aif360.algorithms.postprocessing.reject_option_classification import (
    RejectOptionClassification,
)


#### 3.1.1 Reuse the first Logistic Regression learn to find the best threshold that maximises its balanced accuracy on the validation dataset

In [50]:
# Find the best classification threshold of the validation dataset
df_val_pred = dataset_orig_panel19_val.copy(deepcopy=True)
df_val_pred.scores = model.predict_proba(dataset_orig_panel19_val.features)[:,1].reshape(-1,1)
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = df_val_pred.scores > class_thresh
    df_val_pred.labels[fav_inds] = df_val_pred.favorable_label
    df_val_pred.labels[~fav_inds] = df_val_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_panel19_val,
                                             df_val_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

In [51]:
f" best indice {best_ind}, corresponding balanced accuracy {ba_arr[best_ind]}, and threshold {best_class_thresh}"

' best indice 26, corresponding balanced accuracy 0.778138731707699, and threshold 0.2673737373737374'

#### 3.1.2 Use the RejectOptionClassification  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [52]:
metric_name = "Statistical parity difference"
metric_ub = 0.05
metric_lb = -0.05

ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)

ROC = ROC.fit(dataset_orig_panel19_val, df_val_pred)

In [53]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.1981
Optimal ROC margin = 0.0849


In [54]:
df_roc_val_pred = ROC.predict(df_val_pred)
get_metrics(
    y_true=y_val,
    y_pred=df_roc_val_pred.labels[:,0],
    prot_attr=df_val_pred.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.03941403450831321),
 'disparate_impact_ratio': 0.8860683791942646,
 'base_rate_preds': np.float64(0.33054875903384817),
 'equal_opportunity_difference': 0.040959657046303555,
 'average_odds_difference': 0.037069489594593885,
 'conditional_demographic_disparity': np.float64(-0.009274892312833162),
 'smoothed_edf': np.float64(0.12096113305336464),
 'df_bias_amplification': np.float64(-0.5901756179743529),
 'balanced_accuracy_score': 0.7686831187962461}

In [55]:
accuracy_score(y_val, df_roc_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_roc_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.7776645020353395, 0.7686831187962461)

#### 3.1.3 Do the same while starting from the Logistic Regression learned on the Reweighted dataset

In [56]:
# Find the best classification threshold of the validation dataset
df_val_pred_rw = dataset_orig_panel19_val.copy(deepcopy=True)
df_val_pred_rw.scores = model_rw.predict_proba(dataset_orig_panel19_val.features)[:,1].reshape(-1,1)
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = df_val_pred_rw.scores > class_thresh
    df_val_pred_rw.labels[fav_inds] = df_val_pred_rw.favorable_label
    df_val_pred_rw.labels[~fav_inds] = df_val_pred_rw.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_panel19_val,
                                             df_val_pred_rw, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

In [57]:
f" best indice {best_ind}, corresponding balanced accuracy {ba_arr[best_ind]}, and threshold {best_class_thresh}"

' best indice 23, corresponding balanced accuracy 0.772509430826847, and threshold 0.23767676767676768'

In [58]:
ROC_rw = ROC.fit(dataset_orig_panel19_val, df_val_pred_rw)

In [59]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC_rw.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC_rw.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.1981
Optimal ROC margin = 0.0162


In [60]:
df_roc_val_pred_rw = ROC.predict(df_val_pred_rw)
get_metrics(
    y_true=y_val,
    y_pred=df_roc_val_pred_rw.labels[:,0],
    prot_attr=df_val_pred_rw.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred_rw.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.04521374283341156),
 'disparate_impact_ratio': 0.8739890065412739,
 'base_rate_preds': np.float64(0.34114655451968756),
 'equal_opportunity_difference': 0.03326708596015693,
 'average_odds_difference': 0.030453342202646116,
 'conditional_demographic_disparity': np.float64(-0.010474977876198086),
 'smoothed_edf': np.float64(0.13468746106527973),
 'df_bias_amplification': np.float64(-0.5764492899624378),
 'balanced_accuracy_score': 0.7688763176438125}

In [61]:
accuracy_score(y_val, df_roc_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_roc_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.7717297349507508, 0.7688763176438125)

#### 3.2 Use the Calibrated Equalised Odds  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

In [62]:
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from tqdm import tqdm

# Learn parameters to equalize odds and apply to create a new dataset
# cost constraint of fnr will optimize generalized false negative rates, that of
# fpr will optimize generalized false positive rates, and weighted will optimize
# a weighted combination of both
cost_constraint = "fnr" # "fnr", "fpr", "weighted"
cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint=cost_constraint,
                                     seed=42)
cpp = cpp.fit(dataset_orig_panel19_val, df_val_pred)

In [63]:
df_ceqodds_val_pred = cpp.predict(df_val_pred)
get_metrics(
    y_true=y_val,
    y_pred=df_ceqodds_val_pred.labels[:,0],
    prot_attr=df_val_pred.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.02918610408073935),
 'disparate_impact_ratio': 0.6942182543380743,
 'base_rate_preds': np.float64(0.08404684235731487),
 'equal_opportunity_difference': 0.06522029850494027,
 'average_odds_difference': 0.02819188110283477,
 'conditional_demographic_disparity': np.float64(-0.01974212632292331),
 'smoothed_edf': np.float64(0.3649686491174231),
 'df_bias_amplification': np.float64(-0.34616810191029446),
 'balanced_accuracy_score': 0.6203454334235002}

In [64]:
accuracy_score(y_val, df_ceqodds_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_ceqodds_val_pred.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.818967946129252, 0.6203454334235002)

In [65]:
cpp_rw = cpp.fit(dataset_orig_panel19_val, df_val_pred_rw)

In [66]:
df_ceqodds_val_pred_rw = cpp_rw.predict(df_val_pred_rw)
get_metrics(
    y_true=y_val,
    y_pred=df_ceqodds_val_pred_rw.labels[:,0],
    prot_attr=df_val_pred_rw.protected_attributes[:, sens_ind],
    pos_label=1,
    sample_weight=df_val_pred_rw.instance_weights,
)

{'base_rate_truth': np.float64(0.2138698873472541),
 'statistical_parity_difference': np.float64(-0.06549596392187172),
 'disparate_impact_ratio': 0.6051655183493769,
 'base_rate_preds': np.float64(0.14029807811094752),
 'equal_opportunity_difference': 0.007035005711165887,
 'average_odds_difference': -0.0059817655781470025,
 'conditional_demographic_disparity': np.float64(-0.028276612743995587),
 'smoothed_edf': np.float64(0.5022531118574891),
 'df_bias_amplification': np.float64(-0.20888363917022845),
 'balanced_accuracy_score': 0.6886436054359004}

In [67]:
accuracy_score(y_val, df_ceqodds_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights), balanced_accuracy_score(y_val, df_ceqodds_val_pred_rw.labels[:,0], sample_weight=dataset_orig_panel19_val.instance_weights)

(0.8327093673467971, 0.6886436054359004)